## Python 1 Project
#### IE MBD Apr 2024
#### Group 4

In [1]:
import pandas as pd

In [ ]:
def read(path: str, db_config: dict = None):
    """
    This function should read a filepath/DB and return a dataframe
    """

In [ ]:
def dataShape(df: pd.Dataframe) -> None:
    """
    Examine the shape of the data, as well as relevant information
    """

    df.info()
    return

In [ ]:
def nanCheck(df: pd.DataFrame) -> pd.DataFrame:
    """
    Check for an deal with NaNs
    """

In [ ]:
def processFiles(paths: list) -> tuple:
    """ 
    Read files from the relevant paths and use functions above to clean
    """

In [ ]:
def joinFrames(df_list: tuple) -> pd.DataFrame:
    """
    Join DF's together
    """

In [ ]:
def write(df: pd.DataFrame, dbConfig: dict = None) -> pd.DataFrame:
    """
    Write to CSV/DB
    """

In [ ]:
def plot(df: pd.DataFrame) -> None:
    """
    Plot relevant data from the DF
    """

In [ ]:
def main() -> None:
    lst = ["path1", "path2"]
    clean_dfs = processFiles(lst)
    merged_dfs = joinFrames(clean_dfs)
    write(merged_dfs)